In [1]:
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf          

## Colocar la direccion de la carpeta de iamgenes originales
path_o= '/home/fer/MATLAB_1/DenoisingNetwork/originals/'
path_r = '/home/fer/MATLAB_1/DenoisingNetwork/restauradas/'

def main():                                                                                                             
    print('Hola Luis Fernando')                                                                                         
                                                                                                                        
def data(path_o):
    training_data = []
    for img in os.listdir(path_o):
        pic_o = cv2.imread(os.path.join(path_o,img),0)
        pic_o = cv2.resize(pic_o,(688,688))
        training_data.append([pic_o])
    training = np.array(training_data).reshape(len(os.listdir(path_o)),pic_o.shape[0],pic_o.shape[1])
    return training, training
                                                                                                                      
def show_figure(x_test):  
    ## Generar una figura vacia
    fig, axs = plt.subplots(1, 5)                                                                                                                                                                              
    plt.gray()                                                                                                          
    a=0                                                                                                                 
                                                                                                 
    for j in range(5):    
        ## Graficar en cada posicon del subplot
        axs[j].imshow(tf.squeeze(x_test[a]))                                                                                                                                              
        a = a+1  
            
def show_one(x_test,i):
    ## Funcion para verificar solo una Imagen
    fig, axs = plt.subplots(1, 1)  
    fig.tight_layout(pad=-1)                                                                                            
    plt.gray()  
    axs.imshow(tf.squeeze(x_test[i]))  
    

def normalization(train, test):
    ## Normalizacion de los valores se colocan entre 0-1
    train_n = train.astype('float32')/255
    test_n = test.astype('float32')/255
    train_n = train_n[...,tf.newaxis]
    test_n = test_n[...,tf.newaxis]
    return train_n, test_n

def restauration(data):
    data_r = data*255/1
    return data_r

def noisy(train, test, noisy_factor=0.1):
    ## Aumentor ruido artificial a las imagenes
    x_train_noisy = train + noisy_factor*tf.random.normal(shape = train.shape)
    x_test_noisy = test + noisy_factor*tf.random.normal(shape = test.shape)
    
    x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.) 
    x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)
    return x_train_noisy, x_test_noisy

def plot_noisy(original, noisy,n=5):
    plt.figure(figsize=(20,8))
    plt.gray()
    
    for i in range(n):
        ax = plt.subplot(2, n , i+1)
        plt.title("Original",size=20)
        plt.imshow(tf.squeeze(original[i]))
        plt.gray()
        bx = plt.subplot(2, n, n+ i +1) 
        plt.title("original + noise", size=20) 
        plt.imshow(tf.squeeze(noisy[i])) 
    plt.show()

In [2]:
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, Input

class NoiseReducer(tf.keras.Model): 
  def __init__(self):

    super(NoiseReducer, self).__init__() 

    self.encoder = tf.keras.Sequential([ 
        Input(shape=(688, 688, 1)), 
 
    Conv2D(32, (3,3), activation='relu', padding='same', strides=2),
    Conv2D(16, (3,3), activation='relu', padding='same', strides=2), 
    Conv2D(8, (3,3), activation='relu', padding='same', strides=2)]) 
    
    self.decoder = tf.keras.Sequential([ 
        Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'), 
        Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
        Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'), 

        Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='same')]) 
  
  def call(self, x): 
    encoded = self.encoder(x) 
    decoded = self.decoder(encoded) 
    return decoded

In [ ]:
x_train, x_test =data(path_o)
x_test

In [ ]:
show_figure(x_test)

In [ ]:
show_one(x_test,2)

In [ ]:
x_train_n, x_test_n =normalization(x_train, x_test)

In [ ]:
x_train_n.shape

In [ ]:
show_one(x_train_n,0)

In [ ]:
x_train_noisy, x_test_noisy =noisy(x_train_n, x_test_n,0.4)

In [ ]:
show_one(x_train_noisy,0)

In [ ]:
plot_noisy(x_train_n,x_train_noisy)

In [ ]:
## Generacion de la clase de la red neuronal
autoencoder = NoiseReducer()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
autoencoder.fit(x_train_noisy, 
                x_train_n, 
                epochs=600, 
                shuffle=True, 
                validation_data=(x_test_noisy, x_test_n))

In [ ]:
encoded_imgs=autoencoder.encoder(x_test_noisy).numpy()
decoded_imgs=autoencoder.decoder(encoded_imgs)
decoded_imgs.shape

In [ ]:

n = 1
plt.figure(figsize=(20, 7))
plt.gray()
for i in range(n): 
  # display original + noise 
    bx = plt.subplot(3, n, i + 1) 
    plt.title("original + noise") 
    plt.imshow(tf.squeeze(x_test_noisy[i])) 
    bx.get_xaxis().set_visible(False) 
    bx.get_yaxis().set_visible(False) 

    # display reconstruction 
    cx = plt.subplot(3, n, i + n + 1) 
    plt.title("reconstructed") 
    plt.imshow(tf.squeeze(decoded_imgs[i])) 
    cx.get_xaxis().set_visible(False) 
    cx.get_yaxis().set_visible(False) 

    # display original 
    ax = plt.subplot(3, n, i + 2*n + 1) 
    plt.title("original") 
    plt.imshow(tf.squeeze(x_test[i])) 
    ax.get_xaxis().set_visible(False) 
    ax.get_yaxis().set_visible(False) 

plt.show()

In [ ]:
show_one(decoded_imgs,0)

In [ ]:
show_one(x_test_n,0)

In [ ]:
## convertir de tensor a una numpy
restauracion = decoded_imgs.numpy()

## regresar a escala de 0 255
norm_image = cv2.normalize(restauracion, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

## convertir al tipo de dato inicial
norm_image = norm_image.astype(np.uint8)


In [ ]:
cv2.imwrite(os.path.join(path_r , '1.jpg'), norm_image[0,:,:,0])

In [5]:
from tensorflow import random_

TypeError: extract_image_patches_v2() missing 5 required positional arguments: 'images', 'sizes', 'strides', 'rates', and 'padding'

In [10]:
a=tf.random_normal_initializer(1234)

In [14]:
print(a.mean)

1234
